In [5]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import cv2
import copy
import tqdm
import glob
import keras
import pickle
import logging
import pandas as pd
import tensorflow as tf

In [6]:
logger = tf.get_logger()
logger.setLevel(logging.ERROR) # or logging.INFO, logging.WARNING, etc.

In [7]:
INT_S1_DIR = sorted(glob.glob("_intermediate/stage1_plantdoc/*"))[-1]
print(INT_S1_DIR)
INT_S2_DIR = "_intermediate/stage2_plantdoc"

#DATASET_DIR = "_data/plant_pathology"
DATASET_DIR = "_data/plantdoc_csv"

PROB_THRESHOLD = .5

_intermediate/stage1_plantdoc/2024_11_19_09_28_17


In [8]:
os.makedirs(INT_S2_DIR, exist_ok=True)

In [9]:
train_data = pd.read_csv(os.path.join(DATASET_DIR, "data.csv"))

In [10]:
def get_latest_checkpoint(dir):
    if not os.path.isdir(dir):
        return None
    ckpts = [file for file in os.listdir(dir) if file.endswith("keras")]
    ckpts.sort()
    return os.path.join(dir, ckpts[-1])

In [11]:
with open(os.path.join(INT_S1_DIR, "total_data.pkl"), 'rb') as file:
    stage1_results = pickle.load(file)

In [12]:
stage1_results.keys()

dict_keys(['SAM + YOLOv8', 'SAM + ResNet'])

In [13]:
stage2_names = ["InceptionV3", "VGG19", "AlexNet", "ResNet152V2"]

In [14]:
# Load best performing model by type
stage2_dict = {
    name: keras.models.load_model(f"../disease_detection/out/best{name}_rgb.keras")
    for name in stage2_names 
}

In [ ]:
import numpy as np

BATCH_SIZE = 16  # Adjust this value based on your GPU memory and model size
stage2_results = {}

for stage2_name, stage2_model in stage2_dict.items():
    for stage1_name, stage1_result in stage1_results.items():
        stage_name = f'({stage1_name}) + ({stage2_name})'
        print(f'Evaluating {stage_name}')
        stage2_results[stage_name] = copy.deepcopy(stage1_result)

        # Iterate through the results in batches
        for index, data in tqdm.tqdm(stage2_results[stage_name].items(), desc=stage_name):
            # Extract the 'healthy' status
            healthy = train_data.iloc[index]['healthy']

            # Collect patches for the current index
            masks = data['masks']
            patches = [mask['patch'] for mask in masks]

             # Ensure all patches are resized to the same shape
            resized_patches = [
                tf.image.resize(patch, (224, 224)).numpy() for patch in patches
            ]

            # Convert resized patches into a tensor
            batch_inputs = tf.convert_to_tensor(resized_patches, dtype=tf.float32)

            batch_inputs = tf.expand_dims(batch_inputs, axis=0) if len(batch_inputs.shape) == 3 else batch_inputs
            
            # Process in mini-batches
            num_patches = len(patches)
            batch_predictions = []
            for start in range(0, num_patches, BATCH_SIZE):
                end = start + BATCH_SIZE
                batch = batch_inputs[start:end]
                predictions = stage2_model(batch).numpy()
                batch_predictions.extend(predictions)

            # Update the predictions and mask probabilities
            data['predictions'] = []
            for mask, prob in zip(masks, batch_predictions):
                prob_diseased = prob.squeeze()[1]  # Assuming second index is for 'diseased'
                data['predictions'].append(prob_diseased)
                mask['prob_diseased'] = prob_diseased
                mask['predicted_sick'] = int(prob_diseased > PROB_THRESHOLD)

Evaluating (SAM + YOLOv8) + (InceptionV3)


(SAM + YOLOv8) + (InceptionV3):   0%|          | 0/512 [00:00<?, ?it/s]WARNING: All log messages before absl::InitializeLog() is called are written to STDERR
W0000 00:00:1732009888.531885 1139946 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732009888.545620 1139946 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732009888.545996 1139946 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732009888.548713 1139946 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732009888.549627 1139946 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732009888.550638 1139946 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732009888.551718 1139946 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0

Evaluating (SAM + ResNet) + (InceptionV3)


(SAM + ResNet) + (InceptionV3): 100%|██████████| 512/512 [00:11<00:00, 45.54it/s]


Evaluating (SAM + YOLOv8) + (VGG19)


(SAM + YOLOv8) + (VGG19):   0%|          | 0/512 [00:00<?, ?it/s]W0000 00:00:1732010571.573509 1139946 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732010571.574001 1139946 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732010571.574368 1139946 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732010571.574737 1139946 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732010571.575100 1139946 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732010571.575467 1139946 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732010571.575824 1139946 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1732010571.576226 1139946 gpu_timer.cc:114] Skipping the delay kernel, measurement 

In [14]:
stage2_results.keys()

dict_keys(['(SAM + YOLOv8) + (InceptionV3)', '(SAM + ResNet) + (InceptionV3)', '(SAM + YOLOv8) + (VGG19)', '(SAM + ResNet) + (VGG19)', '(SAM + YOLOv8) + (AlexNet)', '(SAM + ResNet) + (AlexNet)', '(SAM + YOLOv8) + (ResNet152V2)', '(SAM + ResNet) + (ResNet152V2)'])

In [15]:
for name, data in stage2_results.items():
    os.makedirs(os.path.join(INT_S2_DIR, name), exist_ok=True)
    with open(os.path.join(INT_S2_DIR, name, "data.pkl"), 'wb+') as file:
        pickle.dump(data, file, protocol=pickle.HIGHEST_PROTOCOL)

In [16]:
with open(os.path.join(INT_S2_DIR, "total.pkl"), "wb+") as file:
    pickle.dump(stage2_results, file, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
print(stage2_results.keys())

first_key1 = next(iter(stage2_results.keys()))
print(stage2_results[first_key1].keys())

first_key2 = next(iter(stage2_results[first_key1].keys()))
print(stage2_results[first_key1][first_key2]['masks'][0].keys())